In [ ]:
from matplotlib.pylab import (figure, semilogx, loglog, xlabel, ylabel, legend, 
                           title, subplot, show, grid)
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import pandas as pd
import sklearn.linear_model as lm
from sklearn import model_selection
import torch
import sys
sys.path.append('../Tools')
from toolbox_02450 import rlr_validate, correlated_ttest, train_neural_net, draw_neural_net

from data_preprocessing import *
dfjoint, dfRec, dfClas = dataPreprocess()

colNamesMeans, colNamesStd, colNamesExt, colNamesOther = getSpecificColNames()
mean = dfjoint.loc[:,colNamesMeans]
sde = dfjoint.loc[:,colNamesStd]
worst = dfjoint.loc[:,colNamesExt]
colNames = colNamesMeans + colNamesStd + colNamesExt

time_discretized = []
for i in range(len(dfRec.iloc[:,0])):
    if dfRec['time'].iloc[i] <= 12:
        time_discretized.append('<1 years')
    elif dfRec['time'].iloc[i] <= 36:
        time_discretized.append('1-3 years')
    elif dfRec['time'].iloc[i] <= 72:
        time_discretized.append('>3-6 years')
    else:
        time_discretized.append('6+ years')

dfRec['time_discretized'] = time_discretized

In [2]:
from matplotlib.pylab import (figure, semilogx, loglog, xlabel, ylabel, legend, 
                           title, subplot, show, grid)
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import pandas as pd
import sklearn.linear_model as lm
from sklearn import model_selection
import torch
import sys
sys.path.append('../Tools')
from toolbox_02450 import rlr_validate, correlated_ttest, train_neural_net, draw_neural_net

from data_preprocessing import *
dfjoint, dfRec, dfClas = dataPreprocess()

colNamesMeans, colNamesStd, colNamesExt, colNamesOther = getSpecificColNames()
mean = dfjoint.loc[:,colNamesMeans]
sde = dfjoint.loc[:,colNamesStd]
worst = dfjoint.loc[:,colNamesExt]
colNames = colNamesMeans + colNamesStd + colNamesExt

time_discretized = []
for i in range(len(dfRec.iloc[:,0])):
    if dfRec['time'].iloc[i] <= 12:
        time_discretized.append('<1 years')
    elif dfRec['time'].iloc[i] <= 36:
        time_discretized.append('1-3 years')
    elif dfRec['time'].iloc[i] <= 72:
        time_discretized.append('>3-6 years')
    else:
        time_discretized.append('6+ years')

dfRec['time_discretized'] = time_discretized

In [3]:
# Data preprocessing
attributeNames = [u'Offset'] + colNames
X = dfRec.loc[:, colNames]

classLabels = dfRec.loc[:,"time_discretized"]
classNames = sorted(set(dfRec.loc[:,"time_discretized"]))
classDict = dict(zip(classNames, range(4)))
                 
y = np.asarray([classDict[value] for value in classLabels])

N = X.shape[0]
M = len(attributeNames)
# M = M + 1
C = len(classNames)

X = np.concatenate((np.ones((X.shape[0],1)),X),1)


In [12]:
## 2 level cross validation, linear regression model

# Crossvalidation
# Create crossvalidation partition for evaluation
K = 10
CV = model_selection.KFold(K, shuffle=True)
#CV = model_selection.KFold(K, shuffle=False)

# Values of lambda
lambdas = np.power(10.,range(-5,9))

# Initialize variables
#T = len(lambdas)
Error_train = np.empty((K,1))
Error_test = np.empty((K,1))
Error_train_rlr = np.empty((K,1))
Error_test_rlr = np.empty((K,1))
Error_train_nofeatures = np.empty((K,1))
Error_test_nofeatures = np.empty((K,1))
w_rlr = np.empty((M,K))
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
w_noreg = np.empty((M,K))
errors_lin_reg = np.empty((K,1))

k=0

In [5]:
print(' ')
print('------------------------------------')
print('Linear regression model:')
print(' ')
k=0
for train_index, test_index in CV.split(X,y):
    
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    internal_cross_validation = 10    
    
    opt_val_err, opt_lambda, mean_w_vs_lambda, train_err_vs_lambda, test_err_vs_lambda = rlr_validate(X_train, y_train, lambdas, internal_cross_validation)

    # Standardize outer fold based on training set, and save the mean and standard
    # deviations since they're part of the model (they would be needed for
    # making new predictions) - for brevity we won't always store these in the scripts
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)
    
    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :] ) / sigma[k, :] 
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :] ) / sigma[k, :]
    
    # Save generalization error for later statistics
    errors_lin_reg[k] = opt_val_err
    
    k+=1
    
    print('Outer fold {0}'.format(k))
    print('has optimal lambda: {0}'.format(opt_lambda))
    print('and optimal generalization error: {0}'.format(opt_val_err))
    

 
------------------------------------
Linear regression model:
 
Outer fold 1
has optimal lambda: 100000000.0
and optimal generalization error: 1.3242008403009602
Outer fold 2
has optimal lambda: 1000.0
and optimal generalization error: 1.3890844917004554
Outer fold 3
has optimal lambda: 10000.0
and optimal generalization error: 1.2749842939922542
Outer fold 4
has optimal lambda: 100000000.0
and optimal generalization error: 1.4338812908882228
Outer fold 5
has optimal lambda: 100000000.0
and optimal generalization error: 1.2917535265096134


In [6]:
#%% 2 level Cross validation, baseline

# Create crossvalidation partition for evaluation
K = 10
K_i = 10
CV = model_selection.KFold(K, shuffle=True)
#CV = model_selection.KFold(K, shuffle=False)

# Initialize variables
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
error_i = np.empty((K_i,1))
error_o = np.empty((K,1))


print(' ')
print('------------------------------------')
print('Baseline model:')
print(' ')
k=0
for train_index, test_index in CV.split(X,y):
    
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    
    CV_i = model_selection.KFold(K_i, shuffle=True)
    
    k_i = 0
    for train_index, test_index in CV_i.split(X_train,y_train):
        
        # extract training and test set for current inner CV fold
        X_train_i = X_train[train_index]
        y_train_i = y_train[train_index]
        X_test_i = X_train[test_index]
        y_test_i = y_train[test_index]
        
        mean_i = np.mean(y_train_i)
        error_i[k_i]=(1/len(y_test_i))*np.sum((y_test_i-mean_i)**2)
        #rykke error her op og find 10 inder errors
        #Error_i[k_i] = (1/len(y_test_i)*
        
        k_i+=1
    
    error_o[k] = min(error_i)
        
    # Standardize outer fold based on training set, and save the mean and standard
    # deviations since they're part of the model (they would be needed for
    # making new predictions) - for brevity we won't always store these in the scripts
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)
    
    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :] ) / sigma[k, :] 
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :] ) / sigma[k, :] 
    
    print('Outer fold {} generalization error: {}'.format(k+1,error_o[k]))

    k+=1

 
------------------------------------
Baseline model:
 
Outer fold 1 generalization error: [0.89246364]
Outer fold 2 generalization error: [0.63545455]
Outer fold 3 generalization error: [0.91340909]
Outer fold 4 generalization error: [0.85687273]
Outer fold 5 generalization error: [0.89707783]
Outer fold 6 generalization error: [0.8124]
Outer fold 7 generalization error: [0.82547166]
Outer fold 8 generalization error: [0.91293333]
Outer fold 9 generalization error: [1.06334495]
Outer fold 10 generalization error: [0.6436713]


In [10]:
# ANN

# Create crossvalidation partition for evaluation
K = 10
K_i = 10
CV = model_selection.KFold(K, shuffle=True)
#CV = model_selection.KFold(K, shuffle=False)

# Initialize variables
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
error_i = np.empty((K_i,1))

minerror=np.empty((K,1))
min_ind=np.empty((K,1))


k_o=1
for train_index, test_index in CV.split(X,y):
    
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]


    # Parameters for neural network classifier
    # K-fold crossvalidation
    CV = model_selection.KFold(K_i, shuffle=True)
    
    #print('Training model of type:\n\n{}\n'.format(str(model())))
    errors = np.empty((K,1)) # make a list for storing generalizaition error in each loop
    for (k, (train_index, test_index)) in enumerate(CV.split(X_train,y_train)): 
        #print('\nCrossvalidation fold: {0}/{1}'.format(k+1,K_i))  
        n_hidden_units = k+1      # number of hidden units
        n_replicates = 1        # number of networks trained in each k-fold
        max_iter = 10000
        
        # Define the model
        model = lambda: torch.nn.Sequential(
                            torch.nn.Linear(M, n_hidden_units), #M features to n_hidden_units
                            torch.nn.Tanh(),   # 1st transfer function,
                            torch.nn.Linear(n_hidden_units, 1), # n_hidden_units to 1 output neuron
                            # no final tranfer function, i.e. "linear output"
                            )
        loss_fn = torch.nn.MSELoss() # notice how this is now a mean-squared-error loss
        
        # Extract training and test set for current CV fold, convert to tensors
        X_train_i = torch.Tensor(X_train[train_index,:])
        y_train_i = torch.Tensor(y_train[train_index])
        X_test_i = torch.Tensor(X_train[test_index,:])
        y_test_i = torch.Tensor(y_train[test_index])
        
        # Train the net on training data
        net, final_loss, learning_curve = train_neural_net(model,
                                                           loss_fn,
                                                           X=X_train_i,
                                                           y=y_train_i,
                                                           n_replicates=n_replicates,
                                                           max_iter=max_iter)
        
        print('\n\tBest loss: {}\n'.format(final_loss))
        
        # Determine estimated class labels for test set
        y_test_est = net(X_test_i)
        
        errors[k]=final_loss # store error rate for current CV fold 
    
    minerror[k_o-1]=min(errors)
    min_ind[k_o-1] = np.argmin(errors)
    k_o += 1


print(' ')
print('------------------------------------')
print('ANN model:')
print(' ')    
for ii in range(10):
    print('For outer fold {} best generalization error is {} for h = {} '.format(ii+1,minerror[ii],min_ind[ii]+1))



	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\ProgramData\Anaconda3\envs\py10\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([99])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		Final loss:
		167	1.3096805	9.1021576e-07

	Best loss: 1.309680461883545


	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\ProgramData\Anaconda3\envs\py10\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	1.3661934	3.141225e-06
		Final loss:
		1073	1.3660035	9.599543e-07

	Best loss: 1.3660035133361816


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		942	1.2900808	9.240443e-07

	Best loss: 1.2900807857513428


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		193	1.2739145	7.486167e-07

	Best loss: 1.2739144563674927


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		439	1.3139387	9.979924e-07

	Best loss: 1.3139387369155884


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		60	1.3500383	9.713084e-07

	Best loss: 1.3500382900238037


	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.2933842	2.9493835e-06
		2000	1.2886734	4.255232e-06
		3000	1.2826508	4.925784e-06
		4000	1.276919	3.734265e-06
		5000	1.2731925	2.0598604e-06
		Final loss:
		5414	1.2723883	9.3689306e-07

	Best loss: 1.2723883390426636


	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.267461	5.5491328e-06
		2000	1.2598943	5.6770773e-06
		3000	1.2542394	3.0414333e-06
		Final loss:
		3764	1.

c:\ProgramData\Anaconda3\envs\py10\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([101])) that is different to the input size (torch.Size([101, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	1.3125222	3.3604986e-06
		Final loss:
		1072	1.3123243	9.992201e-07

	Best loss: 1.3123242855072021


	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.333724	1.4300904e-06
		Final loss:
		1025	1.3336796	9.832204e-07

	Best loss: 1.3336795568466187


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		709	1.3414763	9.775058e-07

	Best loss: 1.341476321220398


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		803	1.2760105	9.342335e-07

	Best loss: 1.276010513305664


	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.3169731	0.000315264
		Final loss:
		1531	1.2727193	9.366494e-07

	Best loss: 1.2727192640304565


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		873	1.2958142	7.359648e-07

	Best loss: 1.2958141565322876


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		75	1.3450928	3.545012e-07

	Best loss: 1.3450927734375


	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.3219547	9.91932e-06
		Final loss:
		1550	1.3187706	9.943358e-07

	Best loss: 1.318770

In [13]:
import sys
print(sys.path)

['c:\\Users\\Jakob Højgaard\\OneDrive - Danmarks Tekniske Universitet\\DTU\\8. Semester\\02450 Introduction to Machine Learning and Data Mining\\project_1\\02450intro_machine_learning\\Project 2\\Scripts', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\python310.zip', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\DLLs', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib', 'c:\\ProgramData\\Anaconda3\\envs\\py10', '', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\win32', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\Pythonwin', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Jakob Højgaard\\.ipython', '../Tools']


In [13]:
#%% Statistical test for method setup II

# MSE are saved in variable:
    # errors_lin_reg for linear regression
    # error_o for baseline
    # minerror for ANN

# Generalization error differences
r_lin_base = errors_lin_reg - error_o
r_lin_ANN = minerror - errors_lin_reg
r_ANN_base = minerror - error_o

# significance level
alpha = 0.05

rho = 1/K

print(' ')
# Test lin reg vs baseline
print('Correlated t-test for linear regression and baseline')
p_val, conf_int = correlated_ttest(r_lin_base, rho, alpha)
print('p-value {} and confidence interval {}'.format(p_val, conf_int))
print(' ')

# Test ANN vs baseline
print('Correlated t-test for ANN and baseline')
p_val, conf_int = correlated_ttest(r_ANN_base, rho, alpha)
print('p-value {} and confidence interval {}'.format(p_val, conf_int))
print(' ')

# Test lin reg vs ANN
print('Correlated t-test for linear regression and ANN')
p_val, conf_int = correlated_ttest(r_lin_ANN, rho, alpha)
print('p-value {} and confidence interval {}'.format(p_val, conf_int))

 
Correlated t-test for linear regression and baseline
p-value 0.012789825807308003 and confidence interval (-1.1167083413544194, -0.1739114710822698)
 
Correlated t-test for ANN and baseline
p-value 2.165393111676037e-05 and confidence interval (0.29755289921442724, 0.5312778098179999)
 
Correlated t-test for linear regression and ANN
p-value 0.0002782461866609748 and confidence interval (0.6424045789944546, 1.4770459424746618)
